# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Retrieve the environment variable
PRICE_DATA = os.getenv("PRICE_DATA")
print("PRICE_DATA:", PRICE_DATA)


PRICE_DATA: C:\\Users\\TazeenQ\\production\\05_src\\data\\prices


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
from dotenv import load_dotenv
import os
from glob import glob

# Write your code below.
# Load the .env file
load_dotenv()

# Retrieve the PRICE_DATA environment variable
PRICE_DATA = os.getenv("PRICE_DATA")

# Find the path of all Parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)

# Print the list of Parquet file paths to confirm
#print("Parquet files:", parquet_files)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [11]:
import dask.dataframe as dd
from dotenv import load_dotenv
import os
from glob import glob

# Load environment variables
load_dotenv()
PRICE_DATA = os.getenv("PRICE_DATA")

# Get all Parquet files, filtering to include only actual files
parquet_files = [f for f in glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True) if os.path.isfile(f)]

# Load all parquet files into a Dask DataFrame
ddf = dd.read_parquet(parquet_files)

# Step 1: Add lags for Close and Adj Close
ddf['Close_lag'] = ddf['Close'].shift(1)
ddf['Adj_Close_lag'] = ddf['Adj Close'].shift(1)

# Step 2: Calculate returns based on Adjusted Close
ddf['returns'] = (ddf['Adj Close'] / ddf['Adj_Close_lag']) - 1

# Step 3: Calculate high-low range
ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

# Assign the processed data to dd_feat
dd_feat = ddf

# Display the result (optional)
dd_feat.head()


Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.532219,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,NaN,8.270744
A,2000-01-04 00:00:00+00:00,40.206825,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,43.532219,-0.076389,2.950645
A,2000-01-05 00:00:00+00:00,37.712791,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,40.206825,-0.062030,4.425964
A,2000-01-06 00:00:00+00:00,36.276836,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,37.712791,-0.038076,2.771820
A,2000-01-07 00:00:00+00:00,39.299934,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,36.276836,0.083334,4.962444


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [12]:
# Write your code below.
# Step 1: Convert the Dask DataFrame to a Pandas DataFrame
df = dd_feat.compute()

# Step 2: Add a rolling average return calculation with a window of 10 days
df['rolling_avg_return'] = df['returns'].rolling(10).mean()

# Display the result (optional)
print(df.head())



Price                       Date  Adj Close      Close       High        Low  \
ticker                                                                         
A      2000-01-03 00:00:00+00:00  43.532219  51.502148  56.464592  48.193848   
A      2000-01-04 00:00:00+00:00  40.206825  47.567955  49.266811  46.316166   
A      2000-01-05 00:00:00+00:00  37.712791  44.617310  47.567955  43.141991   
A      2000-01-06 00:00:00+00:00  36.276836  42.918453  44.349072  41.577251   
A      2000-01-07 00:00:00+00:00  39.299934  46.494991  47.165592  42.203148   

Price        Open   Volume       sector                       subsector  year  \
ticker                                                                          
A       56.330471  4674353  Health Care  Life Sciences Tools & Services  2000   
A       48.730328  4765083  Health Care  Life Sciences Tools & Services  2000   
A       47.389126  5758642  Health Care  Life Sciences Tools & Services  2000   
A       44.080830  2534434  Health

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
>No, it was not strictly necessary to convert to Pandas, as Dask also supports rolling operations. However, using Pandas allows for easier manipulation and faster computation on smaller datasets.
+ Would it have been better to do it in Dask? Why?
>It would have been better to do it in Dask if the dataset is large and doesn't fit into memory, as Dask can handle larger-than-memory computations and parallel processing across multiple cores.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.